# Deploy to Kubernetes in Google Cloud

In [ ]:
gcloud auth list
gcloud config list project
gcloud config set compute/region us-central1

export PROJECT_ID=$(gcloud config get-value project)
export IMAGE_NAME="valkyrie-app"
export TAG_NAME="v0.0.2"
export REPOSITORY_NAME="valkyrie-repository"

### Task 1. Create a Docker image and store the Dockerfile

In [ ]:
source <(gsutil cat gs://cloud-training/gsp318/marking/setup_marking_v2.sh)
gcloud source repos clone valkyrie-app

cd ~/marking/valkyrie-app
cat > Dockerfile <<EOF
FROM golang:1.10
WORKDIR /go/src/app
COPY source .
RUN go install -v
ENTRYPOINT ["app","-single=true","-port=8080"]
EOF

docker build -t $IMAGE_NAME:$TAG_NAME .

bash ~/marking/step1_v2.sh

### Task 2. Test the created Docker image

In [ ]:
cd ~/marking/valkyrie-app
docker run -p 8080:8080 $IMAGE_NAME:$TAG_NAME &

bash ~/marking/step2_v2.sh

### Task 3. Push the Docker image to the Artifact Registry

In [ ]:
cd ~/marking/valkyrie-app
gcloud auth configure-docker us-central1-docker.pkg.dev

gcloud artifacts repositories create $REPOSITORY_NAME \
--repository-format=docker \
--location=us-central1 

docker build -t us-central1-docker.pkg.dev/$PROJECT_ID/$REPOSITORY_NAME/$IMAGE_NAME:$TAG_NAME .

docker push us-central1-docker.pkg.dev/$PROJECT_ID/$REPOSITORY_NAME/$IMAGE_NAME:$TAG_NAME

### Task 4. Create and expose a deployment in Kubernetes

In [ ]:
gcloud container clusters get-credentials valkyrie-dev \
--zone=us-east1-d

sed -i s#IMAGE_HERE#us-central1-docker.pkg.dev/$PROJECT_ID/$REPOSITORY_NAME/$IMAGE_NAME:$TAG_NAME#g ~/marking/valkyrie-app/k8s/deployment.yaml

kubectl create -f ~/marking/valkyrie-app/k8s/deployment.yaml
kubectl create -f ~/marking/valkyrie-app/k8s/service.yaml